In [2]:
import numpy as np
import pandas as pd
import psycopg2
import getpass
# for configuring connection 
from configobj import ConfigObj
import os


In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [3]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
csv = '/home/mei/nas/docker/thesis/data/csv/'

## flat_feature table: 
- select patient who comes from direct and emergency and the stay length in icu between 1 and 14 days

In [8]:
create_table_query =  query_schema +"""

drop materialized view if exists flat cascade;
create materialized view flat as
SELECT DISTINCT patientunitstayid, extract(hour from to_timestamp(unitadmittime24,'HH24:MI:SS')) as hour, gender, age, apacheadmissiondx, admissionweight,dischargeweight, unitdischargelocation,unitdischargestatus
FROM patient
WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
    AND unitdischargeoffset >=1440 
    AND unitdischargeoffset <=20160
group by patientunitstayid, hour
order by patientunitstayid, hour
"""

In [9]:
with con.begin() as connection:
    connection.execute(text(create_table_query))
select_query = "SELECT * FROM flat;"
df_flat = pd.read_sql_query(select_query, con)

In [10]:
df_flat

,patientunitstayid,hour,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,141168,15.0,Female,70,"Rhythm disturbance (atrial, supraventricular)",84.30,85.8,Death,Expired
1,141265,14.0,Male,67,"CVA, cerebrovascular accident/stroke",100.00,91.8,Floor,Alive
2,141266,21.0,Male,73,"Sepsis, renal/UTI (including bladder)",120.40,112.9,Floor,Alive
3,141276,14.0,Female,59,"Arrest, respiratory (without cardiac arrest)",156.60,156.6,Home,Alive
4,141284,23.0,Male,63,Anemia,NaN,88.5,Floor,Alive
...,...,...,...,...,...,...,...,...,...
68441,3353147,17.0,Male,24,"Overdose, sedatives, hypnotics, antipsychotics...",153.40,151.3,Floor,Alive
68442,3353194,10.0,Female,51,Cardiac arrest (with or without respiratory ar...,63.05,65.8,Death,Expired
68443,3353213,7.0,Female,51,Coma/change in level of consciousness (for hep...,54.40,59.4,Home,Alive
68444,3353226,18.0,Female,79,"Effusions, pleural",58.40,55.1,Death,Expired


##  labels
- unitdischarge location, unitdiscahrges status, actualiculos

In [11]:
create_table_query =  query_schema +"""
drop materialized view if exists labels cascade;
create materialized view labels as
  -- select all the data we need from the apache predictions table, plus patient identifier and hospital identifier
  -- information because we only want to select one episode per patient (more on this later)
  with all_labels as (
    select p.uniquepid, p.patienthealthsystemstayid, apr.patientunitstayid, p.unitvisitnumber,
      apr.predictedhospitalmortality, apr.actualhospitalmortality, apr.predictediculos, apr.actualiculos
      from patient as p
      inner join apachepatientresult as apr
        on p.patientunitstayid = apr.patientunitstayid
      -- only use the most recent apache prediction model and exclude anyone who doesn't have at least 24 hours of data
      where apr.apacheversion = 'IVa' and apr.actualiculos >= 1
    )
  select al.patientunitstayid, al.predictedhospitalmortality, al.actualhospitalmortality,
    al.predictediculos, al.actualiculos
    from all_labels as al
    -- 'selection' is a table which will choose a random hospital stay (the lowest number is fine because the stays
    -- are randomly ordered). In the case of multiple ICU stays within that hospital admission, it will choose the
    -- first ICU stay that satisfies the 24 hours of data requirement. The rationale is that the model should be
    -- applied as soon as there is 24 hours of continuous data within the hospital. This query extracts 89143 stays.
    inner join (
      select p.uniquepid, p.patienthealthsystemstayid, min(p.unitvisitnumber) as unitvisitnumber
        from patient as p
        inner join (
          select uniquepid, min(patienthealthsystemstayid) as patienthealthsystemstayid
            from all_labels
            group by uniquepid
          ) as intermediate_selection
          on p.patienthealthsystemstayid = intermediate_selection.patienthealthsystemstayid
        group by p.uniquepid, p.patienthealthsystemstayid
      ) as selection
      on al.patienthealthsystemstayid = selection.patienthealthsystemstayid
      and al.unitvisitnumber = selection.unitvisitnumber;
    inner join flat as f on la.patientunitstayid = f.patientunitstayid;
"""

In [12]:
select_query = "SELECT * FROM labels;"
df_labels = pd.read_sql_query(select_query, con)

In [13]:
df_labels

,patientunitstayid,predictedhospitalmortality,actualhospitalmortality,predictediculos,actualiculos
0,141168,5.9098619924959253E-2,EXPIRED,3.091127,2.4972
1,141194,0.1022830270275107,ALIVE,4.167129,3.3423
2,141203,0.4709733472175876,ALIVE,8.670299,1.2979
3,141227,0.48856191407785921,ALIVE,8.209624,1.1472
4,141233,3.5490436381315972E-2,ALIVE,3.107540,10.8923
...,...,...,...,...,...
89138,3353147,4.8307271745665827E-2,ALIVE,5.498694,1.0888
89139,3353190,-1,ALIVE,-1.000000,2.1187
89140,3353194,0.86952467986787907,EXPIRED,7.674351,2.4930
89141,3353213,5.2999142347007012E-2,ALIVE,5.801727,3.4118


In [14]:
flat_index = df_flat.set_index('patientunitstayid').index

In [15]:
df_labels = df_labels[df_labels['patientunitstayid'].isin(flat_index)]
col_labels = ['patientunitstayid','actualiculos']
col_flats = ['patientunitstayid','unitdischargelocation','unitdischargestatus']
df_labels = df_labels[col_labels].merge(df_flat[col_flats], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [16]:
df_labels

,patientunitstayid,actualiculos,unitdischargelocation,unitdischargestatus
0,141168,2.4972,Death,Expired
1,141265,4.2138,Floor,Alive
2,141266,1.0423,Floor,Alive
3,141276,1.1694,Home,Alive
4,141284,1.4416,Floor,Alive
...,...,...,...,...
51332,3353123,1.0409,Death,Expired
51333,3353140,1.3243,Skilled Nursing Facility,Alive
51334,3353147,1.0888,Floor,Alive
51335,3353194,2.4930,Death,Expired


## diagnosis

In [6]:
create_table_query =  query_schema +"""
drop materialized view if exists diagnoses cascade;
create materialized view diagnoses as

  -- for past medical history:
  select ph.patientunitstayid, ph.pasthistorypath as diagnosisstring
    from pasthistory as ph
    inner join labels as l on l.patientunitstayid = ph.patientunitstayid
    inner join flat as f on f.patientunitstayid = ph.patientunitstayid
    where ph.pasthistoryoffset > 0
    and ph.pasthistorypath LIKE '%Organ Systems%'
       
"""

In [7]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 
    
select_query = "SELECT * FROM diagnoses;"
df_dx = pd.read_sql_query(select_query, con)

In [8]:
df_dx

,patientunitstayid,diagnosisstring
0,141168,notes/Progress Notes/Past History/Organ System...
1,141168,notes/Progress Notes/Past History/Organ System...
2,141168,notes/Progress Notes/Past History/Organ System...
3,141168,notes/Progress Notes/Past History/Organ System...
4,141168,notes/Progress Notes/Past History/Organ System...
...,...,...
220555,3353194,notes/Progress Notes/Past History/Organ System...
220556,3353194,notes/Progress Notes/Past History/Organ System...
220557,3353194,notes/Progress Notes/Past History/Organ System...
220558,3353213,notes/Progress Notes/Past History/Organ System...


## medication

In [18]:
query = query_schema + """
select a.patientunitstayid, a.drugname
from admissiondrug a
inner join flat f on a.patientunitstayid = f.patientunitstayid
where a.drugoffset > 0
order by a.patientunitstayid
"""

In [19]:
df_drug = pd.read_sql_query(query,con)

In [30]:
df_drug

,patientunitstayid,drugname
0,252784,MULTIVITAMIN ...
1,252784,NITROGLYCERIN ...
2,252784,ASPIRIN ...
3,252784,LISINOPRIL ...
4,252784,LANTUS ...
...,...,...
324439,3348105,OMEPRAZOLE ...
324440,3348105,GLIMEPIRIDE ...
324441,3348105,METFORMIN HCL ...
324442,3348105,TRIAMTERENE W/HCTZ ...


## lab_ ts

In [21]:
create_table_query =  query_schema +"""
drop materialized view if exists timeserieslab cascade;
create materialized view timeserieslab as
  select l.patientunitstayid, l.labresultoffset, l.labname
    from lab as l
    inner join commonlabs as cl
      on cl.labname = l.labname  -- only include the common labs
    inner join labels as la
      on la.patientunitstayid = l.patientunitstayid -- only extract data for the cohort
    where l.labresultoffset > 0;

"""

In [22]:
select_query = "SELECT * FROM timeserieslab;"
df_timeserieslab = pd.read_sql_query(select_query, con)

In [23]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
0,141168,1133,PT - INR,2.50
1,141168,1133,PT,26.60
2,141168,231,PT - INR,1.70
3,141168,516,BUN,26.00
4,141168,231,PT,17.10
...,...,...,...,...
6814785,3353216,787,MCHC,32.00
6814786,3353216,787,MCH,30.00
6814787,3353216,787,Hct,26.00
6814788,3353216,787,RBC,2.79


## vital periodic

In [24]:
create_table_query =  query_schema +"""
drop materialized view if exists timeseriesperiodic cascade;
create materialized view timeseriesperiodic as
  select vp.patientunitstayid, vp.observationoffset, vp.temperature, vp.sao2, vp.heartrate, vp.respiration, vp.cvp,
    vp.systemicsystolic, vp.systemicdiastolic, vp.systemicmean, vp.st1, vp.st2, vp.st3
    from vitalperiodic as vp
    -- select only the patients who are in the cohort
    inner join labels as la
      on la.patientunitstayid = vp.patientunitstayid
    where vp.observationoffset > 0
    order by vp.patientunitstayid, vp.observationoffset;
"""

In [25]:
select_query = "SELECT * FROM timeseriesperiodic;"
df_timeseriesperiodic = pd.read_sql_query(select_query, con)

In [26]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
0,141168,119,NaN,93.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,124,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,129,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,134,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,139,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24337939,3353216,1417,NaN,95.0,85.0,25.0,NaN,166.0,105.0,120.0,0.0,0.0,0.0
24337940,3353216,1422,NaN,96.0,83.0,22.0,NaN,160.0,102.0,117.0,0.0,0.0,0.0
24337941,3353216,1427,NaN,96.0,86.0,23.0,NaN,163.0,101.0,116.0,0.0,0.0,0.0
24337942,3353216,1432,NaN,94.0,95.0,23.0,NaN,159.0,79.0,101.0,0.0,0.0,0.0


In [28]:
# df_dx=pd.read_csv(csv+'/diagnoses.csv')
# df_timeserieslab=pd.read_csv(csv+'/timeserieslab.csv')
# df_timeseriesperiodic=pd.read_csv(csv+'/timeseriesperiodic.csv')
# df_labels=pd.read_csv(csv+'/labels.csv')
# df_flat=pd.read_csv(csv+'/flat.csv')


In [32]:
common_ids = set(df_labels['patientunitstayid'].unique()) \
             .intersection(df_dx['patientunitstayid'].unique()) \
             .intersection(df_timeserieslab['patientunitstayid'].unique()) \
             .intersection(df_timeseriesperiodic['patientunitstayid'].unique()) \
             .intersection(df_flat['patientunitstayid'].unique())\
             .intersection(df_drug['patientunitstayid'].unique())

In [38]:
common_ids_df = pd.DataFrame(list(common_ids), columns=['patientunitstayid'])
common_ids_df.to_csv(csv+'/common_ids.csv', index=False)

In [10]:
# common_ids=pd.read_csv(csv+'/common_ids.csv')

In [33]:
len(common_ids)

9928

In [34]:
def filter_and_sort(dataframes, common_ids, id_column='patientunitstayid'):
    """
    Filter and sort dataframes by common
    """
    return [
        df[df[id_column].isin(common_ids)].sort_values(by=id_column)
        for df in dataframes
    ]

In [35]:
dataframes = [df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat, df_drug]
filtered_sorted_dataframes = filter_and_sort(dataframes, common_ids)

# back to the original dataframes
df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat,df_drug = filtered_sorted_dataframes

In [36]:
def export_table_to_csv(df, output_file):
    """
    Export a SQL query to a CSV file
    """
    df.to_csv(output_file, index=False)
    print(f"Exported {output_file}")


In [46]:
## labels
export_table_to_csv(df_labels, f"{csv}labels.csv")
## static data
export_table_to_csv(df_dx, f"{csv}diagnoses.csv")
export_table_to_csv(df_flat, f"{csv}flat.csv")
export_table_to_csv(df_drug, f"{csv}drug.csv")

## time series data
export_table_to_csv(df_timeserieslab, f"{csv}timeserieslab.csv")
export_table_to_csv(df_timeseriesperiodic, f"{csv}timeseriesperiodic.csv")


Exported /home/mei/nas/docker/thesis/data/csv/labels.csv
Exported /home/mei/nas/docker/thesis/data/csv/diagnoses.csv
Exported /home/mei/nas/docker/thesis/data/csv/flat.csv
Exported /home/mei/nas/docker/thesis/data/csv/drug.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeserieslab.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeseriesperiodic.csv


In [39]:
df_flat

,patientunitstayid,hour,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,252784,15.0,Male,56,Diabetic ketoacidosis,75.00,75.7,Floor,Alive
1,253331,1.0,Male,76,"CHF, congestive heart failure",114.70,114.4,Floor,Alive
2,255112,5.0,Male,52,Cardiac arrest (with or without respiratory ar...,50.60,58.2,Death,Expired
3,258354,19.0,Female,61,"Hypertension, uncontrolled (for cerebrovascula...",74.38,75.3,Floor,Alive
4,259414,1.0,Male,81,"CVA, cerebrovascular accident/stroke",107.50,106.7,Floor,Alive
...,...,...,...,...,...,...,...,...,...
10491,3247116,21.0,Female,52,"Arrest, respiratory (without cardiac arrest)",61.70,68.0,Death,Expired
10492,3247360,21.0,Female,25,"Sepsis, renal/UTI (including bladder)",48.90,60.3,Floor,Alive
10493,3247421,1.0,Male,59,"Sepsis, pulmonary",76.20,82.1,Floor,Alive
10494,3346588,17.0,Female,71,"CHF, congestive heart failure",121.50,115.3,Telemetry,Alive


In [40]:
df_labels

,patientunitstayid,actualiculos,unitdischargelocation,unitdischargestatus
0,252784,2.0500,Floor,Alive
1,253331,1.7625,Floor,Alive
2,255112,10.7381,Death,Expired
3,258354,3.0090,Floor,Alive
4,259414,5.8562,Floor,Alive
...,...,...,...,...
10491,3247116,8.1256,Death,Expired
10492,3247360,3.0097,Floor,Alive
10493,3247421,2.9520,Floor,Alive
10494,3346588,2.1826,Telemetry,Alive


In [42]:
df_dx

,patientunitstayid,diagnosisstring
7823,252784,notes/Progress Notes/Past History/Organ System...
7824,252784,notes/Progress Notes/Past History/Organ System...
7825,252784,notes/Progress Notes/Past History/Organ System...
7936,253331,notes/Progress Notes/Past History/Organ System...
7935,253331,notes/Progress Notes/Past History/Organ System...
...,...,...
219673,3347496,notes/Progress Notes/Past History/Organ System...
219672,3347496,notes/Progress Notes/Past History/Organ System...
219671,3347496,notes/Progress Notes/Past History/Organ System...
219677,3347496,notes/Progress Notes/Past History/Organ System...


In [43]:
df_drug

,patientunitstayid,drugname
0,252784,MULTIVITAMIN ...
10,252784,OMEPRAZOLE ...
9,252784,CARVEDILOL ...
8,252784,NOVOLOG ...
7,252784,OMEGA 3 ...
...,...,...
324394,3347496,XANAX ...
324395,3347496,CYMBALTA ...
324396,3347496,ATENOLOL ...
324398,3347496,CLOPIDOGREL BISULFATE ...


In [44]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
443685,252784,1080,bedside glucose,109.000
443780,252784,1020,phosphate,2.000
443781,252784,1240,bicarbonate,18.000
443782,252784,1240,Hgb,10.600
443783,252784,-160,MCH,30.800
...,...,...,...,...
6777891,3347496,55,calcium,8.800
6777892,3347496,55,MCH,29.000
6777893,3347496,55,ALT (SGPT),13.000
6777895,3347496,229,pH,7.454


In [45]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
1676074,252784,10,NaN,100.0,106.0,22.0,NaN,NaN,NaN,NaN,0.05,0.05,0.0
1676269,252784,985,NaN,99.0,84.0,16.0,NaN,NaN,NaN,NaN,-0.20,0.00,0.2
1676268,252784,980,NaN,99.0,85.0,16.0,NaN,NaN,NaN,NaN,-0.10,0.00,0.1
1676267,252784,975,NaN,99.0,86.0,14.0,NaN,NaN,NaN,NaN,-0.10,0.00,0.2
1676266,252784,970,NaN,99.0,85.0,14.0,NaN,NaN,NaN,NaN,0.00,0.00,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24203276,3347496,512,NaN,98.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24203275,3347496,507,NaN,98.0,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24203274,3347496,502,NaN,98.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24203280,3347496,532,NaN,98.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## get the common ids in all the processed tables

In [ ]:
diagnoses = pd.read_csv(csv+'/preprocessed_diagnoses.csv')
flat_drug = pd.read_csv(csv+'/preprocessed_flat_drug.csv')
flat = pd.read_csv(csv+'/preprocessed_flat.csv')
labels = pd.read_csv(csv+'/preprocessed_labels.csv')
timeseries = pd.read_csv(csv+'/preprocessed_timeseries.csv',low_memory=False)

In [14]:
timeseries.rename(columns={'Unnamed: 1': 'time'}, inplace=True)

In [16]:
flat_drug.drop(columns=['hour'], inplace=True)

In [18]:
common_ids = set(labels['patient'].unique())\
               .intersection(diagnoses['patient'].unique())\
                .intersection(flat_drug['patient'].unique())\
                .intersection(flat['patient'].unique())\
                .intersection(timeseries['patient'].unique())

In [25]:
print(diagnoses['patient'].dtype)
print(flat_drug['patient'].dtype)
print(flat['patient'].dtype)
print(labels['patient'].dtype)
print(timeseries['BUN'].dtype)

int64
int64
int64
int64
object


In [22]:
len(common_patients_df)

0

In [19]:
len(common_ids)

0

In [50]:
def filter_and_sort_patient(dataframes, common_ids, id_column='patient'):
    """
    Filter and sort dataframes by common
    """
    return [
        df[df[id_column].isin(common_ids)].sort_values(by=id_column)
        for df in dataframes
    ]

In [54]:
dataframes = [labels, diagnoses, flat_drug, flat, timeseries]
filtered_sorted_dataframes = filter_and_sort_patient(dataframes, common_ids)

# back to the original dataframes
labels, diagnoses, flat_drug, flat, timeseries= filtered_sorted_dataframes

In [62]:
## labels
export_table_to_csv(labels, f"{csv}/final_labels.csv")

## static data
export_table_to_csv(diagnoses, f"{csv}/final_diagnoses.csv")
export_table_to_csv(flat_drug, f"{csv}/final_flat_drug.csv")
export_table_to_csv(flat, f"{csv}/final_flat.csv")

## time series data
export_table_to_csv(timeseries, f"{csv}/final_timeseries.csv")


Exported /home/mei/nas/docker/thesis/data/csv/final_labels.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_diagnoses.csv
Exported /home/mei/nas/docker/thesis/data/csv/_final_flat_drug.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_flat.csv
Exported /home/mei/nas/docker/thesis/data/csv/final_timeseries.csv


In [7]:
labels

,patient,actualiculos,unitdischargestatus,dischargeweight,discharge_risk_category
0,252784,2.0500,0,75.7,1
1,253331,1.7625,0,114.4,1
2,255112,10.7381,1,58.2,4
3,258354,3.0090,0,75.3,1
4,259414,5.8562,0,106.7,1
...,...,...,...,...,...
9814,3247116,8.1256,1,68.0,4
9815,3247360,3.0097,0,60.3,1
9816,3247421,2.9520,0,82.1,1
9817,3346588,2.1826,0,115.3,2


In [8]:
diagnoses

,patient,Cardiovascular (R),Cardiovascular (R)|AICD,Cardiovascular (R)|Angina,Cardiovascular (R)|Arrhythmias,Cardiovascular (R)|Arrhythmias|atrial fibrillation - chronic,Cardiovascular (R)|Arrhythmias|atrial fibrillation - intermittent,Cardiovascular (R)|Congestive Heart Failure,Cardiovascular (R)|Congestive Heart Failure|CHF,Cardiovascular (R)|Congestive Heart Failure|CHF - severity unknown,...,"apacheadmissiondx_Rhythm disturbance (atrial, supraventricular)",apacheadmissiondx_Rhythm disturbance (conduction defect),apacheadmissiondx_Seizures (primary-no structural brain disease),"apacheadmissiondx_Sepsis, GI","apacheadmissiondx_Sepsis, cutaneous/soft tissue","apacheadmissiondx_Sepsis, other","apacheadmissiondx_Sepsis, pulmonary","apacheadmissiondx_Sepsis, renal/UTI (including bladder)","apacheadmissiondx_Sepsis, unknown","grouped_apacheadmissiondx_Overdose,"
0,252784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,253331,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,255112,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,258354,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,259414,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,3247116,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9924,3247360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
9925,3247421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
9926,3346588,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
flat_drug

,patient,gender,age,admissionweight,> 89,ACETAMINOPHEN,ADVAIR DISKUS,ALBUTEROL,ALBUTEROL SULFATE,ALLOPURINOL,...,TRAMADOL HCL,TRAZODONE HCL,TYLENOL,VITAMIN C,VITAMIN D,VITAMIN D3,WARFARIN SODIUM,XANAX,ZOCOR,ZOFRAN
0,252784,1,56.0,75.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,253331,1,76.0,114.7,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
2,255112,1,52.0,50.6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,258354,0,61.0,74.4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,259414,1,81.0,107.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9517,3247094,1,22.0,91.2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9518,3247116,0,52.0,61.7,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9519,3247421,1,59.0,76.2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9520,3346588,0,71.0,121.5,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
timeseries

,patient,time,-basos,-eos,-lymphs,-monos,-polys,ALT (SGPT),AST (SGOT),BUN,...,sao2_mask,heartrate_mask,respiration_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask,st1_mask,st2_mask,st3_mask
0,252784,1,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,252784,2,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,252784,3,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,252784,4,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,252784,5,0.8,0.4,16.6,4.4,0.0,34.0,24.0,23.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6248414,3347496,286,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,0.06818181818181819,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6248415,3347496,287,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,0.0625,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6248416,3347496,288,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,0.057692307692307696,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6248417,3347496,289,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,0.053571428571428575,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
nan_counts = timeseries.isna().sum()
print(nan_counts)

patient                   0
time                      2
-basos                    0
-eos                      0
-lymphs                   0
                         ..
systemicdiastolic_mask    0
systemicmean_mask         0
st1_mask                  0
st2_mask                  0
st3_mask                  0
Length: 116, dtype: int64


In [11]:
flat

,patient,hour,gender,age,admissionweight,> 89
0,252784,15.0,1,56.0,75.0,0
1,253331,1.0,1,76.0,114.7,0
2,255112,5.0,1,52.0,50.6,0
3,258354,19.0,0,61.0,74.4,0
4,259414,1.0,1,81.0,107.5,0
...,...,...,...,...,...,...
9889,3247116,21.0,0,52.0,61.7,0
9890,3247360,21.0,0,25.0,48.9,0
9891,3247421,1.0,1,59.0,76.2,0
9892,3346588,17.0,0,71.0,121.5,0
